In [5]:
import numpy as np
from tensorflow import keras
from keras.datasets import mnist
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

num_capas_ocultas = [1,2,3]
dim_capas_ocultas = [512, 1024]
learningRate = [0.025, 0.01]
batch_size = 128
best_acc=0.0
param = []
for lr in learningRate:
    for n in num_capas_ocultas:
        for d in dim_capas_ocultas:
            model = Sequential()
            model.add(Input(784))
            for numCapas in range(n):
                model.add(Dense(d, activation='relu'))
            model.add(Dense(num_classes, activation='softmax'))

            sgd=SGD(learning_rate=lr, momentum=0.9)

            callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) ## También podría ser 'loss' sin necesitar un conjunto de validación

            model.compile(loss='categorical_crossentropy',
                        optimizer=sgd,
                        metrics=['accuracy'])

            epochs=100  ## No nos preocupemos, el fit acabará antes por el early_stopping
            history = model.fit(x_train, y_train,
                                batch_size=batch_size,
                                epochs=epochs,
                                verbose=1,
                                validation_data=(x_val, y_val),
                                callbacks=[callback])
            if history.history['val_accuracy'][-1]>best_acc:
                best_acc=history.history['val_accuracy'][-1]
                besth=d
                param = "Learning rate: " + str(lr) +" \n Numero de capas ocultas: " + str(n) + "Dimension capas ocultas: " +str(d)


print("=============================")
print("Best acc",best_acc)
print("Best hidden dim",besth)
print("..............Best param........ \n" , param)
print("=============================")

## PROBAR EL MEJOR CON TEST
model = Sequential()
model.add(Input(784))
model.add(Dense(besth, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
sgd=SGD(learning_rate=0.01, momentum=0.9)
callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) ## También podría ser 'loss' sin necesitar un conjunto de validación
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
x_combined = np.concatenate((x_train, x_val), axis=0)
y_combined = np.concatenate((y_train, y_val), axis=0)
history = model.fit(x_combined, y_combined, # <--- aquí podríamos concatenar trainin+valid para no malgastar datos
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[callback])

training set (60000, 28, 28)
test set (10000, 28, 28)
training set (48000, 784)
val set (12000, 784)
Epoch 1/100
375/375 [==============================] - 3s 5ms/step - loss: 0.3867 - accuracy: 0.8896 - val_loss: 0.2239 - val_accuracy: 0.9383
Epoch 2/100
375/375 [==============================] - 1s 3ms/step - loss: 0.1913 - accuracy: 0.9460 - val_loss: 0.1632 - val_accuracy: 0.9547
Epoch 3/100
375/375 [==============================] - 1s 3ms/step - loss: 0.1388 - accuracy: 0.9616 - val_loss: 0.1348 - val_accuracy: 0.9615
Epoch 4/100
375/375 [==============================] - 1s 3ms/step - loss: 0.1093 - accuracy: 0.9698 - val_loss: 0.1120 - val_accuracy: 0.9686
Epoch 5/100
375/375 [==============================] - 1s 4ms/step - loss: 0.0900 - accuracy: 0.9752 - val_loss: 0.1064 - val_accuracy: 0.9701
Epoch 6/100
375/375 [==============================] - 1s 4ms/step - loss: 0.0752 - accuracy: 0.9797 - val_loss: 0.0955 - val_accuracy: 0.9718
Epoch 7/100
375/375 [====================